# Import Libraries

In [1]:
import numpy as np
import pandas as pd
from hmmlearn import hmm
import warnings
from constants import *
import math
import os
import random
import matplotlib.pyplot as plt
import dill
from functools import reduce

# Helper Methods

In [2]:
def generate_random_sample(X, size):
    '''
    Given a list X, 
    generate random samples of given size
    '''
    Z_temp = random.sample(list(X), size)
    
    #Concatenation
    Z = [Z_temp[0]]
    for val in Z_temp[1:]:
        Z = np.concatenate([Z,[val]])
    
    return Z

def convert_values_to_list(list_val):
    '''
    Given a list X = [1 2 3] , 
    return X = [[1],[2],[3]]
    '''
    X = []
    for i in list_val:
        X.append([i])
    return X

def conversion_list_of_list(X, DIMENSION):
    '''
    Given a list X with values in lists,
    X = [[ 3  2  2],[4 8 10]]
    Convert each value to a list
    Return list of lists, array of lengths of each sequence
    X = [[3] [2] [2] [4] [8] [10]]
    ''' 
    X_new = []
    length = len(X)
    for idx, val_list in enumerate(X):
        Y = []
        for val in val_list:
            Y.append([val])
        X_new.append(Y)

    #Concatenation
    Z = X_new[0]
    for val_list in X_new[1:]:
        Z = np.concatenate([Z,val_list])

    # assign array of lengths for HMM
    lengths = [DIMENSION]*length
    
    return Z,lengths

#Calculate likelihood for given sequence according to given HMMs and return HMM
def likelihood_sequence(sequence, HMM_array):
    '''
    Given list of K HMMs and sequence,
    determines likelihood of sequence under all HMM models
    Returns index of HMM which has max likelihood
    ''' 
    scores = []
    length = [len(sequence)]
    for i, HMM in enumerate(HMM_array):
        calculated_score = HMM.score(sequence, length)
        scores.append(calculated_score)
    idx = scores.index(max(scores))
    return idx

def HMM_model_stats(model):
    '''
    Details of HMM model
    ''' 
    print("*************************************")
    print("Transition matrix")
    print(model.transmat_)
    print("*************************************")
    print("Means and stds of each hidden state")
    for i in range(model.n_components):
        print("Hidden state {0}".format(i))
        print("mean = ", model.means_[i])
        print("std = ", [np.sqrt(model.covars_[i])])
        print()

def BIC(model,X,lengths):
    LogLikelihood = model.score(X,lengths)
    num_hidden_states = model.n_components
    # D counts transition matrix (emission estimated by PDF), means = num_hidden_states  
    # covariance matrix = num_hidden_states
    D = num_hidden_states**2 + 2*num_hidden_states
    BIC = LogLikelihood - (D/2)*np.log(len(X))
    return BIC
    
def BIC_array(HMM_array,X_i):
    BIC_total = 0
    for i in range(len(X_i)):
        model = HMM_array[i]
        X, lengths = conversion_list_of_list(X_i[i],DIMENSION)
        BIC_total+= BIC(model,X,lengths)
    return BIC_total

def likelihood_array(HMM_array,X_i):
    likelihood_total = 1
    for i in range(len(X_i)):
        if(len(X_i[i])>=HMM_array[i].n_components):
            model = HMM_array[i]
            X, lengths = conversion_list_of_list(X_i[i],DIMENSION)
            LogLikelihood = model.score(X,lengths)
            likelihood = LogLikelihood 
            likelihood_total*= likelihood
    return likelihood_total

def plot_BIC(list_k, BIC_score):
    fig = plt.subplot(111)
    plt.plot(list_k, BIC_score, marker='o')  
    plt.xlabel('Value of K')
    plt.ylabel('Objective')
    plt.title('BIC')
    plt.show() 

def print_stats(assignments,length):
    for i in range(length):
        if(i%100==0):
            print(assignments[i])   

# Merge data

In [3]:
previous = 'RPKMOutput/GeneNames/RNASeq/geneTXCD_RPKMoutput_'
timepoints = ['ATCACG-s_6_1_genome.txt','TTAGGC-s_6_1_genome.txt',
             'CAGATC-s_6_1_genome.txt','GCCAAT-s_6_1_genome.txt',
             'CTTGTA-s_6_1_genome.txt'
            ]
dataframes = []
for idx,file in enumerate(timepoints):
    df = pd.read_csv(previous+file, sep="	", na_values=['-'])
    df = df.dropna(how='any')
    df = df[['AccNum', 'GeneName','cdReads',
       'cdRPKM']]
    df.rename(columns={'cdRPKM':'cdRPKM'+str(idx), 'cdReads':'cdReads'+str(idx)}, inplace=True)
    dataframes.append(df)

In [4]:
df_rnaseq = reduce(lambda left,right: pd.merge(left,right,on=['AccNum','GeneName']), dataframes)

In [5]:
df_rnaseq.head()

,AccNum,GeneName,cdReads0,cdRPKM0,cdReads1,cdRPKM1,cdReads2,cdRPKM2,cdReads3,cdRPKM3,cdReads4,cdRPKM4
0,NM_017847,ODR4,50.0,6.642349,62.0,5.591876,71.0,6.004645,30.0,5.369988,51.0,8.507454
1,NM_001143986,TLE6,4.0,0.421957,6.0,0.429708,3.0,0.201468,3.0,0.426413,3.0,0.397381
2,NM_001003803,ATP5S,29.0,8.115351,52.0,9.879314,52.0,9.263817,19.0,7.164127,22.0,7.730539
3,NM_001003800,BICD2,778.0,54.937502,1155.0,55.371390,1444.0,64.913315,706.0,67.172940,883.0,78.293873
4,NM_016649,ESF1,17.0,1.206070,26.0,1.252307,10.0,0.451649,9.0,0.860333,20.0,1.781686


In [6]:
# df_rnaseq.to_csv('RPKMOutput/RNASeq.txt',sep=' ',index=False)

In [7]:
previous = 'RPKMOutput/GeneNames/RPF/geneTXCD_RPKMoutput_'
timepoints = ['ATCACG-s_7_1_genome.txt','TTAGGC-s_7_1_genome.txt',
             'CAGATC-s_7_1_genome.txt','GCCAAT-s_7_1_genome.txt',
             'CTTGTA-s_7_1_genome.txt'
            ]
dataframes = []
for idx,file in enumerate(timepoints):
    df = pd.read_csv(previous+file, sep="	", na_values=['-'])
    df = df.dropna(how='any')
    df = df[['AccNum', 'GeneName','cdReads',
       'cdRPKM']]
    df.rename(columns={'cdRPKM':'cdRPKM'+str(idx), 'cdReads':'cdReads'+str(idx)}, inplace=True)
    dataframes.append(df)
df_rpf = reduce(lambda left,right: pd.merge(left,right,on=['AccNum','GeneName']), dataframes)
df_rpf.head()

,AccNum,GeneName,cdReads0,cdRPKM0,cdReads1,cdRPKM1,cdReads2,cdRPKM2,cdReads3,cdRPKM3,cdReads4,cdRPKM4
0,NM_017847,ODR4,93.0,16.487335,62.0,13.206484,49.0,15.968568,37.0,16.540714,39.0,20.464235
1,NM_001143986,TLE6,4.0,0.563098,2.0,0.338285,2.0,0.517555,1.0,0.354984,1.0,0.416666
2,NM_001003803,ATP5S,81.0,30.248942,83.0,37.241851,71.0,48.740039,46.0,43.317961,26.0,28.738355
3,NM_001003800,BICD2,501.0,47.210965,389.0,44.043594,284.0,49.195554,257.0,61.069347,230.0,64.149959
4,NM_016649,ESF1,69.0,6.532635,52.0,5.915217,41.0,7.135519,26.0,6.207228,26.0,7.285780


In [8]:
# df_rpf.to_csv('RPKMOutput/RPF.txt',sep=' ',index=False)

In [9]:
# df_rnaseq = df_rnaseq[['AccNum','GeneName', 'cdRPKM0', 'cdRPKM1',
#        'cdRPKM2', 'cdRPKM3', 'cdRPKM4']]
# df_rpf = df_rpf[['AccNum','GeneName', 'cdRPKM0', 'cdRPKM1',
#        'cdRPKM2', 'cdRPKM3', 'cdRPKM4']]
df_TE = reduce(lambda left,right: pd.merge(left,right,on=['AccNum','GeneName']), [df_rnaseq,df_rpf])
df_TE.head()

,AccNum,GeneName,cdReads0_x,cdRPKM0_x,cdReads1_x,cdRPKM1_x,cdReads2_x,cdRPKM2_x,cdReads3_x,cdRPKM3_x,...,cdReads0_y,cdRPKM0_y,cdReads1_y,cdRPKM1_y,cdReads2_y,cdRPKM2_y,cdReads3_y,cdRPKM3_y,cdReads4_y,cdRPKM4_y
0,NM_017847,ODR4,50.0,6.642349,62.0,5.591876,71.0,6.004645,30.0,5.369988,...,93.0,16.487335,62.0,13.206484,49.0,15.968568,37.0,16.540714,39.0,20.464235
1,NM_001143986,TLE6,4.0,0.421957,6.0,0.429708,3.0,0.201468,3.0,0.426413,...,4.0,0.563098,2.0,0.338285,2.0,0.517555,1.0,0.354984,1.0,0.416666
2,NM_001003803,ATP5S,29.0,8.115351,52.0,9.879314,52.0,9.263817,19.0,7.164127,...,81.0,30.248942,83.0,37.241851,71.0,48.740039,46.0,43.317961,26.0,28.738355
3,NM_001003800,BICD2,778.0,54.937502,1155.0,55.371390,1444.0,64.913315,706.0,67.172940,...,501.0,47.210965,389.0,44.043594,284.0,49.195554,257.0,61.069347,230.0,64.149959
4,NM_016649,ESF1,17.0,1.206070,26.0,1.252307,10.0,0.451649,9.0,0.860333,...,69.0,6.532635,52.0,5.915217,41.0,7.135519,26.0,6.207228,26.0,7.285780


In [10]:
df_TE[['cdRPKM0_x']] = df_TE[['cdRPKM0_x']].div(df_TE['cdRPKM0_y'].values,axis=0)
df_TE[['cdRPKM1_x']] = df_TE[['cdRPKM1_x']].div(df_TE['cdRPKM1_y'].values,axis=0)
df_TE[['cdRPKM2_x']] = df_TE[['cdRPKM2_x']].div(df_TE['cdRPKM2_y'].values,axis=0)
df_TE[['cdRPKM3_x']] = df_TE[['cdRPKM3_x']].div(df_TE['cdRPKM3_y'].values,axis=0)
df_TE[['cdRPKM4_x']] = df_TE[['cdRPKM4_x']].div(df_TE['cdRPKM4_y'].values,axis=0)
for i in range(0,5):
    df_TE.rename(columns={'cdRPKM'+str(i)+'_x':'TE'+str(i)}, inplace=True)


In [11]:
df_TE.keys()

Index(['AccNum', 'GeneName', 'cdReads0_x', 'TE0', 'cdReads1_x', 'TE1',
       'cdReads2_x', 'TE2', 'cdReads3_x', 'TE3', 'cdReads4_x', 'TE4',
       'cdReads0_y', 'cdRPKM0_y', 'cdReads1_y', 'cdRPKM1_y', 'cdReads2_y',
       'cdRPKM2_y', 'cdReads3_y', 'cdRPKM3_y', 'cdReads4_y', 'cdRPKM4_y'],
      dtype='object')

In [12]:
df_TE = df_TE[['AccNum', 'GeneName','TE0','TE1','TE2','TE3','TE4']]
df_TE.head()

,AccNum,GeneName,TE0,TE1,TE2,TE3,TE4
0,NM_017847,ODR4,0.402876,0.423419,0.376029,0.324653,0.415723
1,NM_001143986,TLE6,0.749349,1.270257,0.389269,1.201215,0.953718
2,NM_001003803,ATP5S,0.268285,0.265275,0.190066,0.165385,0.268997
3,NM_001003800,BICD2,1.163660,1.257195,1.319496,1.099945,1.220482
4,NM_016649,ESF1,0.184622,0.211709,0.063296,0.138602,0.244543


In [13]:
# df_TE.to_csv('RPKMOutput/TE.txt',sep=' ',index=False)

# Load Data and Clean

In [14]:
df_rnaseq.keys()
df_TE.keys()

Index(['AccNum', 'GeneName', 'TE0', 'TE1', 'TE2', 'TE3', 'TE4'], dtype='object')

In [15]:
#Dataset
# df_rnaseq = df_rnaseq[(df_rnaseq['cdReads0'] >= 50) & (df_rnaseq['cdReads1'] >= 50) & (df_rnaseq['cdReads2'] >= 50)& (df_rnaseq['cdReads3'] >= 50)& (df_rnaseq['cdReads4'] >= 50)]
df_main = df_TE[['TE0', 'TE1', 'TE2', 'TE3', 'TE4']]
LENGTH,DIMENSION = df_main.shape
print("Dataset size is",LENGTH)
print("Features are", DIMENSION)
print(df_main.head(5))
X = np.log2(df_main.values)
print("****************************")
print("First 5 log2 values\n",X[:5])

Dataset size is 11631
Features are 5
        TE0       TE1       TE2       TE3       TE4
0  0.402876  0.423419  0.376029  0.324653  0.415723
1  0.749349  1.270257  0.389269  1.201215  0.953718
2  0.268285  0.265275  0.190066  0.165385  0.268997
3  1.163660  1.257195  1.319496  1.099945  1.220482
4  0.184622  0.211709  0.063296  0.138602  0.244543
****************************
First 5 log2 values
 [[-1.31159283 -1.23984226 -1.41108413 -1.62303084 -1.26630538]
 [-0.41629021  0.34512025 -1.36115891  0.26449443 -0.068366  ]
 [-1.89815922 -1.91444197 -2.39542881 -2.59610252 -1.8943366 ]
 [ 0.21866934  0.33020854  0.3999865   0.13743175  0.28745108]
 [-2.43735182 -2.23984226 -3.98174532 -2.85098279 -2.03184012]]


# Arrays with HMM models for 1<=K<=25

In [17]:
HMM_K_ARRAYS = []
X_i_K_ARRAYS = []

# Check likelihood and do assignments

In [20]:
K_values = range(12,16)

In [ ]:
for K in K_values:
    HMM_array = []
    X_i = []
    print("**************** K =", K ,"************************")
    for i in range(K):
        X_i.append([])
        
    NUM_ITERATIONS = 0
    NUM_CLUSTER_PREV = {}
    NUM_CLUSTER_NOW = {}
    
    # Sequences for initial HMM estimation
    # Make K subsets data of LENGTH
    for i in range(LENGTH):
        for j in range(K):
            if(i%K==j):
                X_i[j].append(list(X[i]))
                NUM_CLUSTER_PREV[i] = j
                
    for i in range(K):
        model = hmm.GaussianHMM(n_components=3,covariance_type='diag')
        X_temp, lengths = conversion_list_of_list(X_i[i],DIMENSION)
        model.fit(X_temp, lengths)
        HMM_array.append(model)
    
    likelihood_prev = likelihood_array(HMM_array,X_i)
    print("Likelihood for iteration",NUM_ITERATIONS,"is",likelihood_prev)
    NUM_ITERATIONS+=1
    while (True):
        # Assign all sequences to HMM models

        print("************ Check likelihood of sequence in HMM  *********")
        NUM_CLUSTER_NOW = {}
        for idx,x in enumerate(X):
            sequence = convert_values_to_list(x)
            hmm_index = likelihood_sequence(sequence, HMM_array)
            X_i[hmm_index].append(list(x))
            NUM_CLUSTER_NOW[idx] = hmm_index
        print("************ Checking likelihood done  *********")

        # Re-estimate parameters for new HMMs
        print("************ Re-estimating HMM *********")
        HMM_array_prev = HMM_array
        HMM_array = []
        for i in range(K):
            model = hmm.GaussianHMM(n_components=3,covariance_type='diag')
            if(len(X_i[i])>=model.n_components):
                X_temp, lengths = conversion_list_of_list(X_i[i], DIMENSION)
                model.fit(X_temp, lengths)
                HMM_array.append(model)
            else:
                HMM_array.append(HMM_array_prev[i])
        print("************ Re-estimation done *********")
        likelihood_curr = likelihood_array(HMM_array,X_i)
        print("Likelihood for iteration",NUM_ITERATIONS,"is",likelihood_curr)
        print("*****************************************")

        # if no reassignments, then break
        if ((NUM_CLUSTER_PREV == NUM_CLUSTER_NOW) or NUM_ITERATIONS>40):
            HMM_K_ARRAYS.append(HMM_array)
            X_i_K_ARRAYS.append(X_i)
            break
        else:
            # initialize empty subsets of data for next iteration
            X_i = []
            for i in range(K):
                X_i.append([])

            NUM_CLUSTER_PREV = NUM_CLUSTER_NOW
            print("Num iterations is:", NUM_ITERATIONS)
            NUM_ITERATIONS += 1
            likelihood_prev = likelihood_curr
    print("**********************************************************\n\n")

**************** K = 12 ************************
Likelihood for iteration 0 is 7.17606145927e+44
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Likelihood for iteration 1 is 2.70220961616e+47
*****************************************
Num iterations is: 1
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Likelihood for iteration 2 is 8.5559475514e+40
*****************************************
Num iterations is: 2
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Likelihood for iteration 3 is 6.4433108258e+40
*****************************************
Num iterations is

Likelihood for iteration 27 is 8.05927811009e+37
*****************************************
Num iterations is: 27
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Likelihood for iteration 28 is 8.08488664879e+37
*****************************************
Num iterations is: 28
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Likelihood for iteration 29 is 7.89855854161e+37
*****************************************
Num iterations is: 29
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Likelihood for iteration 30 is 8.09097360257e+37
************************************

************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Likelihood for iteration 13 is -7.07512264777e+40
*****************************************
Num iterations is: 13
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Likelihood for iteration 14 is -5.68643577922e+40
*****************************************
Num iterations is: 14
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Likelihood for iteration 15 is -4.72428978981e+40
*****************************************
Num iterations is: 15
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM **

************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Likelihood for iteration 40 is -8.82206487218e+39
*****************************************
Num iterations is: 40
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Likelihood for iteration 41 is -9.26145454763e+39
*****************************************
**********************************************************


**************** K = 14 ************************
Likelihood for iteration 0 is 2.44681831137e+51
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Likelihood for iteration 1 is 1.57913411709e+54
*****************************************
Num iterations is: 1
*********

Likelihood for iteration 25 is -2.58167511134e+43
*****************************************
Num iterations is: 25
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Likelihood for iteration 26 is -2.19954564163e+43
*****************************************
Num iterations is: 26
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Likelihood for iteration 27 is -1.99628461723e+43
*****************************************
Num iterations is: 27
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Likelihood for iteration 28 is -1.89050834963e+43
********************************

************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Likelihood for iteration 11 is 1.14927823859e+45
*****************************************
Num iterations is: 11
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Likelihood for iteration 12 is 1.14033134467e+45
*****************************************
Num iterations is: 12
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Likelihood for iteration 13 is 1.19459397837e+45
*****************************************
Num iterations is: 13
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM *****

In [ ]:
print(len(HMM_K_ARRAYS))
print(len(X_i_K_ARRAYS))

In [ ]:
dill.dump_session('../Weights/HMM_GaussianHMM_5points_TE.db')

# Calculate BIC for these assignments

In [ ]:
print(BIC_array(HMM_array,X_i))

In [ ]:
print(BIC_array(HMM_array,X_i))

# Noise Cluster

In [ ]:
# Get list of lists to fit
Z, lengths = conversion_list_of_list(X,DIMENSION)

In [ ]:
# Noise cluster 

model_noise = hmm.GMMHMM(n_components=3,n_mix=3,covariance_type='spherical')
model_noise.fit(Z, lengths)

In [ ]:
HMM_model_stats(model_noise)

# Generate subsets of data for K HMMs

In [46]:
K = 3
HMM_array = []
X_i = []

for i in range(K):
    X_i.append([])

# Sequences for initial HMM estimation
for i in range(LENGTH):
    for j in range(K):
        if(i%K==j):
            X_i[j].append(list(X[i]))

In [47]:
for i in range(K):
    model = hmm.GaussianHMM(n_components=3, covariance_type='full')
    X_temp, lengths = conversion_list_of_list(X_i[i],DIMENSION)
    model.fit(X_temp, lengths)

## Statistics for K HMMs

In [48]:
HMM_model_stats(model)
print('\n\n')
model.predict([[7.56030946],[5.96591207],[6.40121497]],lengths=[3])

*************************************
Transition matrix
[[  9.99276869e-01   6.38482140e-04   8.46487741e-05]
 [  1.93144205e-04   9.99806856e-01   2.07287602e-13]
 [  1.68090394e-02   2.12582122e-11   9.83190961e-01]]
*************************************
Means and stds of each hidden state
Hidden state 0
mean =  [ 5.85591189]
std =  [array([[ 0.49587845]])]

Hidden state 1
mean =  [ 7.75447354]
std =  [array([[ 1.05975496]])]

Hidden state 2
mean =  [ 4.32879483]
std =  [array([[ 0.68723547]])]






array([1, 1, 1])

In [ ]:
K = 3
HMM_array = []
X_i = []

for i in range(K):
    X_i.append([])

# Sequences for initial HMM estimation
for i in range(LENGTH):
    for j in range(K):
        if(i%K==j):
            X_i[j].append(list(X[i]))
for i in range(K):
    model = hmm.GaussianHMM(n_components=3, covariance_type='diag')
    X_temp, lengths = conversion_list_of_list(X_i[i],DIMENSION)
    model.fit(X_temp, lengths)
    HMM_array.append(model)
for i in range(K):
    print("Statistics for HMM Model", i)
    HMM_model_stats(HMM_array[i])
    print('\n\n')

In [ ]:
NUM_ITERATIONS = 1
NUM_CLUSTER_PREV = {}
NUM_CLUSTER_NOW = {}

In [ ]:
#initialize empty subsets of data
X_i = []

for i in range(K):
    X_i.append([])

In [ ]:
likelihood_prev = likelihood_array(HMM_array,X_i)
print("Likelihood for iteration",0,"is",likelihood_prev)